In [1]:
# Need to Run
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from lxml import html
import time
from selenium.webdriver.support.ui import Select
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep

In [2]:
url = 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&SchoolID=173897003954&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=173897003954'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
url = 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Bayless+Elementary+School&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=481623001212'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
tables = soup.find_all('table')

In [5]:
len(tables)

18

In [6]:
trial_dict = {}

In [7]:
table = tables[7]
data = []
if table:
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        data.append(cols)

In [8]:
data_dict = {}

In [10]:
testt = []

In [11]:
for x in tables[2].find('tr').find_all('td'):
    testt.append(x.text.strip())
    

for item in testt:
    if 'NCES School ID' in item:
        nces_id = item.split(':')[-1].strip()
        nces_id = int(nces_id)
        data_dict['NCES School ID'] = nces_id
    elif 'State School ID' in item:
        state_id = item.split(':')[-1].strip()
        data_dict['State School ID'] = state_id

In [13]:
testt

['School Name:\nBAYLES EL',
 '',
 'NCES School ID:\n481623001212',
 '',
 'State School ID:\nTX-057905-057905108']

In [14]:
rows = tables[2].find_all('tr')
data_dict = {}
for row in rows:
    columns = row.find_all('td')
    if columns:
        x = columns[0].text.split(":")
        x = [item.strip() for item in x]
        if len(x)>=2:
            key = x[0]
            value = x[1]
            data_dict[key] = value
        else:
            pass

In [15]:
data_dict

{'School Name': 'BAYLES EL',
 'District Name': 'Dallas Isddistrict information',
 'Mailing Address': '2444 TELEGRAPH AVEDALLAS,\xa0TX\xa075228-5819',
 'Type': 'Regular school',
 'Supervisory Union #': 'N/A',
 'Website': 'http'}

In [18]:
from lxml import etree

new_url = 'view-source:https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Bayless+Elementary+School&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=481623001212'

tree = etree.HTML(new_url)

In [19]:
import sys

sys.getsizeof(tree)

56

In [20]:
tree.xpath("//table")

[]

In [22]:
# tree.xpath("/html/body/[@size='2']")

In [23]:
# Need to Run
def fix_grades(lst):
    pk_count = lst.count('PK')
    kg_count = lst.count('KG')
    
    if pk_count > 1:
        while 'PK' in lst:
            lst.remove('PK')
            pk_count -= 1
            if pk_count == 1:
                break
    
    if kg_count > 1:
        while 'KG' in lst:
            lst.remove('KG')
            kg_count -= 1
            if kg_count == 1:
                break
    
    return lst

In [24]:
tree

<Element html at 0x7f2f35f9ea00>

In [27]:
pattern = re.compile(r'\b(PK?|KG?|^\d+$)\b')
pattern = re.compile(r'\b(PK?|KG?|^\d+$)\b')
#pattern = re.compile(r'(?<!\b(PK|KG)\b)(?<!\d)\b(PK|KG|\d+$)\b')
#pattern = re.compile('^\d+$')


# Extract numbers from each string in the list
cleaned_numbers = [match.group() for item in testt for match in re.finditer(pattern, item)]
cleaned_lst = fix_grades(cleaned_numbers)
midpoint = len(cleaned_lst) //2

grade = cleaned_lst[:midpoint]
enrollment = cleaned_lst[midpoint:]
print(grade)
enrollment_dict = {f'Grade {grade}': enrollment[i] for i, grade in enumerate(grade)}


[]


In [28]:
enrollment_dict

{}

In [29]:
print((tables[13].find_all('tr')[1].find_all('td')))

[<td bgcolor="#134F8A" width="100%">
<table bgcolor="#134F8A" border="0" bordercolor="#134F8A" cellpadding="0" cellspacing="1">
<tr>
<td align="right" bgcolor="#23619E"><font color="#FFFFFF" size="2"> </font></td>
<td align="center" bgcolor="#23619E" width="60"><strong><font color="#FFFFFF" size="2">Male</font></strong></td>
<td align="center" bgcolor="#23619E" width="60"><strong><font color="#FFFFFF" size="2">Female</font></strong></td>
</tr>
<tr>
<td align="right" bgcolor="#23619E"><font color="#FFFFFF" size="2"> Students </font></td>
<td align="center" bgcolor="#F5F196" width="60"><font size="2">219</font></td>
<td align="center" bgcolor="#F5F196" width="60"><font size="2">190</font></td>
</tr>
</table>
</td>, <td align="right" bgcolor="#23619E"><font color="#FFFFFF" size="2"> </font></td>, <td align="center" bgcolor="#23619E" width="60"><strong><font color="#FFFFFF" size="2">Male</font></strong></td>, <td align="center" bgcolor="#23619E" width="60"><strong><font color="#FFFFFF" siz

In [30]:
for i in range(len(tables[13].find_all('tr')[0].find_all('td'))):
    datastr = ((tables[13].find_all('tr')[1].find_all('td')[i].text))

data_list = [item.strip() for item in datastr.split('\n') if item.strip()]

genders = data_list[0:2]
student_counts = data_list[3:5]

In [31]:
student_counts

['219', '190']

In [35]:
race_num = []
for i in range(len(tables[12].find_all('tr')[0].find_all('td'))):
    race_num.append((tables[12].find_all('tr')[1].find_all('td')[i].text))

race = []
for i in range(len(tables[12].find_all('tr')[0].find_all('td'))):
    race.append((tables[12].find_all('tr')[0].find_all('td')[i].text))

clean_race = []
for i in race_num:
    clean_race.append(i.strip())



race_num = clean_race[1:]
race = race[1:]

race_data = {race: int(count) for race, count in zip(race, race_num)}

print(race_data)

{'American Indian/Alaska Native': 2, 'Asian': 1, 'Black': 84, 'Hispanic': 306, 'Native Hawaiian/Pacific Islander': 0, 'White': 9, 'Two or MoreRaces': 7}


In [36]:
# don't delete this allows to create list of the numbers
test = []
for i in range(len(tables[7].find_all('tr')[0].find_all('td'))):
    test.append(tables[7].find_all('tr')[0].find_all('td')[i].text)

In [37]:
test2 = []
for i in range(len(tables[7].find('tr').find_all('td'))):
    test2.append(tables[7].find_all('tr')[0].find_all('td')[i].text.strip())

In [38]:
test2

['Enrollment by Grade:\xa0\n\n\n\n\n\n\n\n\n\xa0\nPK\nKG\n1\n2\n3\n4\n5\n\n\n\xa0Students\xa0\n69\n59\n71\n51\n54\n54\n51',
 'Enrollment by Grade:',
 'PK\nKG\n1\n2\n3\n4\n5\n\n\n\xa0Students\xa0\n69\n59\n71\n51\n54\n54\n51',
 'PK\nKG\n1\n2\n3\n4\n5\n\n\n\xa0Students\xa0\n69\n59\n71\n51\n54\n54\n51',
 '',
 'PK',
 'KG',
 '1',
 '2',
 '3',
 '4',
 '5',
 'Students',
 '69',
 '59',
 '71',
 '51',
 '54',
 '54',
 '51']

In [39]:
for row in rows:
    columns = row.find_all('td')
    print(columns[0])

<td valign="top" width="40%"><font size="2"><strong>School Name:<br/>
</strong></font><font size="3">BAYLES EL</font></td>
<td><img border="0" height="4" src="/ccd/commonfiles/images/spacer.gif" width="10"/></td>
<td valign="top" width="40%"><strong><font size="2">District Name:</font></strong><br/>
<font size="3">Dallas Isd</font><br><font size="2"><a href="../districtsearch/district_detail.asp?Search=2&amp;details=1&amp;ID2=4816230&amp;DistrictID=4816230">district information</a></font></br></td>
<td><img border="0" height="10" src="/ccd/commonfiles/images/spacer.gif" width="10"/></td>
<td valign="top" width="40%"><strong><font size="2">Mailing Address:</font></strong><br/><font size="3">2444 TELEGRAPH AVE<br/>DALLAS, TX 75228-5819</font></td>
<td><img border="0" height="8" src="/ccd/commonfiles/images/spacer.gif" width="10"/></td>
<td colspan="2" width="40%"><font size="2"><strong>Type: </strong><br/></font><font size="3">Regular school</font></td>
<td><img border="0" height="8" src

In [40]:
rows = tables[6].find_all('tr')

In [41]:
tables[7].find_all('tr')

[<tr>
 <td align="left" width="100%">
 <table border="0" cellpadding="0" cellspacing="0">
 <tr>
 <td width="100%"><strong><font size="2">Enrollment by Grade:<font size="3"> </font></font></strong></td>
 </tr>
 <tr>
 <td width="100%">
 <table bgcolor="#134F8A" border="0" cellpadding="0" cellspacing="0">
 <tr>
 <td bgcolor="#134F8A">
 <table bgcolor="#134F8A" border="0" bordercolor="#134F8A" cellpadding="0" cellspacing="1">
 <tr>
 <td align="right" bgcolor="#23619E"><font color="#FFFFFF" size="2"> </font></td>
 <td align="center" bgcolor="#23619E" width="31"><strong><font color="#FFFFFF" size="2">PK</font></strong></td>
 <td align="center" bgcolor="#23619E" width="31"><strong><font color="#FFFFFF" size="2">KG</font></strong></td>
 <td align="center" bgcolor="#23619E" width="31"><strong><font color="#FFFFFF" size="2">1</font></strong></td>
 <td align="center" bgcolor="#23619E" width="31"><strong><font color="#FFFFFF" size="2">2</font></strong></td>
 <td align="center" bgcolor="#23619E" wi

In [42]:
urls = ['https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Bayless+Elementary+School&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=481623001212']


In [43]:
urls = ['https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&SchoolID=173897003954&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=173897003954',
        'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Kipp+Indy+Legacy+High+School&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=180020602669',
        'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Bayless+Elementary+School&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=481623001212'
]

In [779]:
# Didn't delete this because I don't know if ill ever neeed it again
# all_data = []
# for i in urls:
#     response = requests.get(i)
#     soup = BeautifulSoup(response.content, 'html.parser')
#     tables = soup.find_all('table')
#     rows = tables[2].find_all('tr')
#     data_dict = {}
#     for row in rows:
#         columns = row.find_all('td')
#         if columns:
#             x = columns[0].text.split(":")
#             x = [item.strip() for item in x]
#             if len(x)>=2:
#                 key = x[0]
#                 value = x[1]
#                 data_dict[key] = value
#             else:
#                 pass
#     # below gets the students, teachers, ratio
#     rows2 = tables[6].find_all('tr')
#     for row in rows2:
#         col_name = row.find_all('td')[0].text.strip()
#         value = row.find_all('td')[2].text.strip()
#         data_dict[col_name] = value
#     nces = []
#     for x in tables[2].find('tr').find_all('td'):
#         nces.append(x.text.strip())


#     for item in testt:
#         if 'NCES School ID' in item:
#             nces_id = item.split(':')[-1].strip()
#             nces_id = int(nces_id)
#             data_dict['NCES School ID'] = nces_id
#         elif 'State School ID' in item:
#             state_id = item.split(':')[-1].strip()
#             data_dict['State School ID'] = state_id
    
#     # getting the grades and their enrollment numbers
#     test = []
#     for i in range(len(tables[7].find('tr').find_all('td'))):
#         test.append(tables[7].find_all('tr')[0].find_all('td')[i].text)
#     #pattern = re.compile('^\d+$')
#     pattern = re.compile(r'\b(PK?|KG?|^\d+$)\b')
#     # Extract numbers from each string in the list
#     cleaned_numbers = [match.group() for item in test for match in re.finditer(pattern, item)]
#     cleaned_numbers = fix_grades(cleaned_numbers)

#     # splitting the list in half because first half is the grade and second half is the value
#     midpoint = len(cleaned_numbers) //2

#     grade = cleaned_numbers[:midpoint]
#     enrollment = cleaned_numbers[midpoint:]
#     for i, grade in enumerate(grade):
#         data_dict[f'Grade {grade}'] = enrollment[i]
    
#     # Getting races and their values
#     race_num = []
#     for i in range(len(tables[12].find_all('tr')[0].find_all('td'))):
#         race_num.append((tables[12].find_all('tr')[1].find_all('td')[i].text))

#     race = []
#     for i in range(len(tables[12].find_all('tr')[0].find_all('td'))):
#         race.append((tables[12].find_all('tr')[0].find_all('td')[i].text))

#     clean_race = []
#     for i in race_num:
#         clean_race.append(i.strip())
#     race_num = clean_race[1:]
#     race = race[1:]
#     #race_data = {race: int(count) for race, count in zip(race, race_num)}
#     for i, race in enumerate(race):
#         data_dict[race] = race_num[i]
#     # Getting genders and     
#     for i in range(len(tables[13].find_all('tr')[0].find_all('td'))):
#         datastr = ((tables[13].find_all('tr')[1].find_all('td')[i].text))

#     data_list = [item.strip() for item in datastr.split('\n') if item.strip()]

#     genders = data_list[0:2]
#     gender_counts = data_list[3:5]
#     for i, gender in enumerate(genders):
#         data_dict[gender] = gender_counts[i]

#     all_data.append(data_dict)

In [ ]:
# pattern = re.compile('^\d+$')
# # Extract numbers from each string in the list
# cleaned_numbers = [match.group() for item in randlist for match in re.finditer(pattern, item)]

# midpoint = len(cleaned_numbers) //2

# grade = cleaned_numbers[:midpoint]
# enrollment = cleaned_numbers[midpoint:]

# enrollment_dict = {f'Grade {grade}': enrollment[i] for i, grade in enumerate(grades)}

In [46]:
#all_data

In [47]:
df = pd.DataFrame.from_dict(all_data)

NameError: name 'all_data' is not defined

In [741]:
df

,School Name,District Name,Mailing Address,Type,Supervisory Union #,Website,Total Students:,Classroom Teachers (FTE):,Student/Teacher Ratio:,Grade 9,...,Two or MoreRaces,Male,Female,Grade PK,Grade KG,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5
0,Thornridge High School,Thornton Twp Hsd 205district information,"15000 Cottage Grove AveDolton, IL 60419-2738",Regular school,N/A,http,"1,057",72.00,14.68,263,...,14,548,509,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KIPP Indy Legacy High,Kipp Indy Legacy Highdistrict information,"2255 N Ralston AveIndianapolis, IN 46218",Regular school,N/A,NaN,377,27.00,13.96,103,...,9,179,198,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BAYLES EL,Dallas Isddistrict information,"2444 TELEGRAPH AVEDALLAS, TX 75228-5819",Regular school,N/A,http,409,32.00,12.78,NaN,...,7,219,190,69,59,71,51,54,54,51


In [48]:
df_master = pd.read_csv('/home/jklingl/capstone/master_school_df.csv')

In [49]:
final_df = pd.read_csv('/home/jklingl/capstone/latest_school_safety_df.csv')

In [50]:
for school in df['School Name']:
    print(school)

NameError: name 'df' is not defined

In [887]:
for school,zc in final_df[['School Name','Zip_Code']].itertuples(index=False):
    print(zc)

46218.0
70346.0
48224.0
21207.0
33626.0
98684.0
60659.0
43605.0
33127.0
28303.0
60419.0
48237.0
80219.0
98122.0
85019.0
30224.0
31907.0
52501.0
37355.0
37874.0
32504.0
20020.0
31636.0
98902.0
70441.0
32780.0
60637.0
70767.0
67152.0
38701.0
90037.0
74106.0
29440.0
30144.0
30236.0
39567.0
32218.0
68701.0
38016.0
98106.0
11204.0
30228.0
21209.0
72143.0
8332.0
31907.0
21221.0
10940.0
48225.0
98122.0
20011.0
97216.0
46222.0
92372.0
37771.0
45405.0
20747.0
60471.0
11413.0
19129.0
89030.0
29456.0
27932.0
48503.0
42003.0
23220.0
30144.0
90059.0
32714.0
85033.0
95212.0
98126.0
45202.0
45237.0
20009.0
96080.0
20832.0
30135.0
46540.0
19143.0
36207.0
75229.0
97386.0
10454.0
49418.0
95632.0
10040.0
10035.0
10029.0
53209.0
44137.0
25951.0
46201.0
21239.0
81006.0
45237.0
71270.0
60644.0
44110.0
21212.0
15208.0
30035.0
80203.0
11206.0
33169.0
19709.0
84770.0
33896.0
85365.0
8094.0
33430.0
5641.0
44105.0
70125.0
40484.0
19720.0
48237.0
43229.0
48205.0
43211.0
37411.0
45416.0
20746.0
63106.0
47885.0
361

In [51]:
final_df['Zip_Code'] = final_df['Zip_Code'].astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [893]:
ul = 'https://nces.ed.gov/ccd/schoolsearch/school_list.asp?Search=1&InstName=Donaldsonville&SchoolID=&Address=&City=&State=&Zip=&Miles=&County=&PhoneAreaCode=&Phone=&DistrictName=&DistrictID=&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1'
response = requests.get(ul)
soup = BeautifulSoup(response.content, 'html.parser')

In [6]:
final_df['Zip_Code'] = final_df['Zip_Code'].fillna('00000')
final_df['Zip_Code'] = final_df['Zip_Code'].astype(str)
final_df['Zip_Code'] = final_df['Zip_Code'].apply(lambda x: '0' + x if len(x) == 4 else x)


In [947]:
final_df['Zip_Code'].describe()

count      1019
unique      864
top       48205
freq          6
Name: Zip_Code, dtype: object

In [52]:
big_data_nces = pd.read_csv('/home/jklingl/capstone/big_data_nces.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/home/jklingl/capstone/big_data_nces.csv'

In [7]:
big_data_nces['Zip_Code'] = big_data_nces['Zip_Code'].astype(str)
big_data_nces['Zip_Code'] = big_data_nces['Zip_Code'].str.pad(width=5, side='left', fillchar='0')

In [18]:
service = Service(executable_path='/home/jklingl/capstone/geckodriver')
options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(service=service, options=options)
driver.get('https://nces.ed.gov/ccd/schoolsearch/index.asp')
dropdown = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/table/tbody/tr[3]/td/table/tbody/tr[3]/td[1]/table/tbody/tr[6]/td/table/tbody/tr[2]/td[5]/table/tbody/tr/td[1]/select')
select = Select(dropdown)
select.select_by_index(6) 

In [44]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from tqdm import tqdm

urls = []
failed_schools = []
total_iterations = len(big_data_nces[['School_left', 'Zip_Code']])

with tqdm(total=total_iterations, desc="Processing", unit="school") as pbar:

    for school, zc in big_data_nces[['School_left', 'Zip_Code']].itertuples(index=False):
        try:
            service = Service(executable_path='/home/jklingl/capstone/geckodriver')
            options = webdriver.FirefoxOptions()
            driver = webdriver.Firefox(service=service, options=options)
            driver.get('https://nces.ed.gov/ccd/schoolsearch/index.asp')
            element = driver.find_element(By.NAME, "InstName")
            element.send_keys(f"{school}")
            element = driver.find_element(By.NAME, "Zip")
            # element.send_keys(f"{zc}")
            button = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/table/tbody/tr[3]/td/table/tbody/tr[3]/td[2]/table/tbody/tr[2]/td/table/tbody/tr/td[2]/input')
            button.click()
            records = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/table[2]/tbody/tr/td/table/tbody/tr/td[2]/font/font/strong')
            records = int(records.text)

            if records == 1:
                time.sleep(1)
                button2 = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/table[3]/tbody/tr[2]/td/table/tbody/tr/td[2]/font/a/strong')
                button2.click()
                time.sleep(1)
                urls.append(driver.current_url)
                driver.close()
            elif records > 1:
                driver.back()
                element = driver.find_element(By.NAME, "Zip")
                element.send_keys(f"{zc}")
                dropdown = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/table/tbody/tr[3]/td/table/tbody/tr[3]/td[1]/table/tbody/tr[6]/td/table/tbody/tr[2]/td[5]/table/tbody/tr/td[1]/select')
                select = Select(dropdown)
                select.select_by_index(3) 
                button = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/table/tbody/tr[3]/td/table/tbody/tr[3]/td[2]/table/tbody/tr[2]/td/table/tbody/tr/td[2]/input')
                button.click()
                records = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/table[2]/tbody/tr/td/table/tbody/tr/td[2]/font/font/strong')
                records = int(records.text)
                if records == 1:
                    time.sleep(1)
                    button2 = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/table[3]/tbody/tr[2]/td/table/tbody/tr/td[2]/font/a/strong')
                    button2.click()
                    time.sleep(1)
                    urls.append(driver.current_url)
                    driver.close()
                elif records > 1:
                    failed_schools.append((school,zc))
                    driver.close()
                    pass
                elif records == 0:
                    failed_schools.append((school,zc))
                    driver.close()
                    pass
            elif records == 0:
                failed_schools.append((school,zc))
                driver.close()
                pass
            
            pbar.update(1)
        except Exception as e:
            print(f"An error occurred for school {school} and zip code {zc}: {str(e)}")
            failed_schools.append((school,zc))
            driver.close()
            continue

    # Now, urls list contains the URLs collected during the iteration
    # failed_schools list contains the schools for which an exception occurred


NameError: name 'big_data_nces' is not defined

In [137]:
# this block is where I am trying to get the failed schools data
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from tqdm import tqdm

urls = []
total_iterations = len(data_list)
failed_schools2 = []
doubles = []

with tqdm(total=total_iterations, desc="Processing", unit="school") as pbar:

    for school, zc in fs:
        try:
            service = Service(executable_path='/home/jklingl/capstone/geckodriver')
            options = webdriver.FirefoxOptions()
            driver = webdriver.Firefox(service=service, options=options)
            driver.get('https://nces.ed.gov/ccd/schoolsearch/index.asp')
            element = driver.find_element(By.NAME, "InstName")
            element.send_keys(f"{school}")
            element = driver.find_element(By.NAME, "Zip")
            # element.send_keys(f"{zc}")
            button = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/table/tbody/tr[3]/td/table/tbody/tr[3]/td[2]/table/tbody/tr[2]/td/table/tbody/tr/td[2]/input')
            button.click()
            records = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/table[2]/tbody/tr/td/table/tbody/tr/td[2]/font/font/strong')
            records = int(records.text)

            if records == 1:
                time.sleep(1)
                button2 = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/table[3]/tbody/tr[2]/td/table/tbody/tr/td[2]/font/a/strong')
                button2.click()
                time.sleep(1)
                urls.append(driver.current_url)
                driver.close()
            elif records > 1:
                driver.back()
                element = driver.find_element(By.NAME, "Zip")
                element.send_keys(f"{zc}")
                dropdown = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/table/tbody/tr[3]/td/table/tbody/tr[3]/td[1]/table/tbody/tr[6]/td/table/tbody/tr[2]/td[5]/table/tbody/tr/td[1]/select')
                select = Select(dropdown)
                select.select_by_index(7) 
                button = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/table/tbody/tr[3]/td/table/tbody/tr[3]/td[2]/table/tbody/tr[2]/td/table/tbody/tr/td[2]/input')
                button.click()
                records = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/table[2]/tbody/tr/td/table/tbody/tr/td[2]/font/font/strong')
                records = int(records.text)
                if records == 1:
                    time.sleep(1)
                    button2 = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/table[3]/tbody/tr[2]/td/table/tbody/tr/td[2]/font/a/strong')
                    button2.click()
                    time.sleep(1)
                    urls.append(driver.current_url)
                    driver.close()
                if records == 2:
                    time.sleep(1)
                    button2 = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/table[3]/tbody/tr[2]/td/table/tbody/tr/td[2]/font/a/strong')
                    button2.click()
                    time.sleep(1)
                    doubles.append(driver.current_url)
                    driver.close()
                elif records > 1:
                    failed_schools2.append((school,zc))
                    driver.close()
                    pass
                elif records == 0:
                    failed_schools2.append((school,zc))
                    driver.close()
                    pass
            elif records == 0:
                failed_schools2.append((school,zc))
                driver.close()
                pass
            
            pbar.update(1)
        except Exception as e:
            print(f"An error occurred for school {school} and zip code {zc}: {str(e)}")
            failed_schools2.append((school,zc))
            driver.close()
            continue

    # Now, urls list contains the URLs collected during the iteration
    # failed_schools list contains the schools for which an exception occurred


Processing: 59school [19:58, 17.99s/school]                                     

An error occurred for school Thunderbolt and zip code 08332: Message: Reached error page: about:neterror?e=dnsNotFound&u=https%3A//nces.ed.gov/ccd/schoolsearch/index.asp&c=UTF-8&d=We%20can%E2%80%99t%20connect%20to%20the%20server%20at%20nces.ed.gov.
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
UnknownError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:800:5
checkReadyState@chrome://remote/content/marionette/navigate.sys.mjs:58:24
onNavigation@chrome://remote/content/marionette/navigate.sys.mjs:324:39
emit@resource://gre/modules/EventEmitter.sys.mjs:154:20
receiveMessage@chrome://remote/content/marionette/actors/MarionetteEventsParent.sys.mjs:33:25



Processing: 102school [34:44, 18.72s/school]

An error occurred for school  and zip code 11218: Alert Text: None
Message: Dismissed user prompt dialog: SEARCH TIP:
This search tool requires at least one of the following 
fields to begin searching for public schools.

  -  School Name
  -  NCES School ID
  -  Street Address
  -  City
  -  State
  -  Zip Code
  -  County
  -  Phone#
  -  District Name
  -  District ID

Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
UnexpectedAlertOpenError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:768:5
GeckoDriver.prototype._handleUserPrompts@chrome://remote/content/marionette/driver.sys.mjs:2809:13



Processing: 133school [44:30, 19.11s/school]

An error occurred for school  and zip code 94124: Alert Text: None
Message: Dismissed user prompt dialog: SEARCH TIP:
This search tool requires at least one of the following 
fields to begin searching for public schools.

  -  School Name
  -  NCES School ID
  -  Street Address
  -  City
  -  State
  -  Zip Code
  -  County
  -  Phone#
  -  District Name
  -  District ID

Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
UnexpectedAlertOpenError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:768:5
GeckoDriver.prototype._handleUserPrompts@chrome://remote/content/marionette/driver.sys.mjs:2809:13



Processing: 775school [3:56:35, 18.32s/school]


In [147]:
len(failed_schools2)

649

In [123]:
fs_copy = failed_schools.copy()

In [148]:
failed_schools2

[('Raleigh', '27610'),
 ('Dub Farris Athletic Complex', '78249'),
 ('Legacy', '90280'),
 ('North Florida Educational Institute', '32220'),
 ('Fairview PreK-6', '45406'),
 ('Cleveland Central Catholic', '44105'),
 ('Darlington-Lee Adult Education Campus', '29010'),
 ('Thibodaux', '70301'),
 ('Savannah', '31404'),
 ('Gilmor', '21213'),
 ('Milwaukee Lutheran', '53222'),
 ('Booker Washington', '61820'),
 ('Lincoln', '20012'),
 ('Premier', '72114'),
 ('Mount Carmel', '60637'),
 ('Brookhaven', '39601'),
 ('O’Bannon', '38701'),
 ('Phoenix', '85029'),
 ('Booker Washington STEM', '61820'),
 ('Margolin Hebrew', '38117'),
 ('Freedom Prep Westwood', '38109'),
 ('John Harris', '17104'),
 ('Julia De Burgos', '19133'),
 ('Milwaukee', '53206'),
 ('Richard Murphy', '02122'),
 ('Oliver Citywide', '15290'),
 ('Rose', '27834'),
 ('Ypsilanti', '48198'),
 ('KIPP College Preparatry', '20024'),
 ('78', '10304'),
 ('Fair View', '19904'),
 ('Chaminade College Preparatory', '91304'),
 ('Theodore Roosevelt', '200

In [149]:
doubles

['https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Hagerstown&Zip=21740&Miles=50&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=240066001288',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Valdosta&Zip=31602&Miles=50&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=130531001734',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Williams&Zip=70510&Miles=50&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=220180001569',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Hamilton&Zip=38106&Miles=50&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=470014801080',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Hetting

In [135]:
def extract_keywords_and_zip(school_info_list):
    """
    Extract keywords from school names and keep the zip codes.

    Args:
    - school_info_list (list): A list of tuples containing the full names of schools and their zip codes.

    Returns:
    - list: A list of tuples containing the modified school names (keywords only) and the original zip codes.
    """
    modified_school_info_list = []

    for school_info in school_info_list:
        school_name, zip_code = school_info

        # Split the school name into words
        words = school_name.split()

        # Define a list of common words to exclude (if necessary)
        # You can expand this list based on your specific requirements
        excluded_words = ['school', 'high', 'elementary', 'middle', 'academy', 'charter', 'stadium','district','county',
                         'community','city','public','(bus)',]

        # Initialize a list to store keywords
        keywords = []

        # Iterate through each word in the school name
        for word in words:
            # Check if the word contains a period
            if '.' not in word:
                # Check if the word is not in the list of excluded words and if it's not a single character
                if word.lower() not in excluded_words and len(word) > 1:
                    # Add the word to the list of keywords
                    keywords.append(word)

        # Join the keywords to form the modified school name
        modified_school_name = ' '.join(keywords)

        modified_school_info_list.append((modified_school_name, zip_code))

    return modified_school_info_list

In [136]:
fs = extract_keywords_and_zip(failed_schools)

In [134]:
fs

[('Raleigh', '27610'),
 ('Dub Farris Athletic Complex', '78249'),
 ('Legacy', '90280'),
 ('North Florida Educational Institute', '32220'),
 ('Fairview PreK-6', '45406'),
 ('Cleveland Central Catholic', '44105'),
 ('Darlington-Lee Adult Education Campus', '29010'),
 ('Thibodaux Stadium', '70301'),
 ('Savannah', '31404'),
 ('Stanton', '20020'),
 ('Gilmor', '21213'),
 ('Milwaukee Lutheran', '53222'),
 ('Booker Washington', '61820'),
 ('Lincoln', '20012'),
 ('Premier', '72114'),
 ('Mount Carmel', '60637'),
 ('Brookhaven', '39601'),
 ('O’Bannon', '38701'),
 ('Phoenix', '85029'),
 ('Josey', '30904'),
 ('Booker Washington STEM', '61820'),
 ('Margolin Hebrew', '38117'),
 ('Marie Reed', '20009'),
 ('Freedom Prep Westwood', '38109'),
 ('John Harris', '17104'),
 ('Japhet', '78210'),
 ('KIPP Troy Prep', '12180'),
 ('Julia De Burgos', '19133'),
 ('Milwaukee', '53206'),
 ('Richard Murphy', '02122'),
 ('Berwick Alternative', '43227'),
 ('Oliver Citywide', '15290'),
 ('Rose', '27834'),
 ('Ypsilanti Co

In [120]:
doubles

['https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Montpelier+&Zip=05641&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=500043900110',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Navarro++&Zip=78758&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=480894012824',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Wasilla+&Zip=99654&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=020051000237',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Drew+Elementary+&Zip=22206&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=510027000087',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstNa

In [96]:
len(doubles)

53

In [99]:
urls.extend(doubles2)

In [100]:
len(urls)

415

In [38]:
import ast

# Read the content of the text file
file_path = '/home/jklingl/capstone/failed_schools.txt'    # Update with your file path
with open(file_path, 'r') as file:
    content = file.read()

# Convert the string representation of tuples into a list of tuples
try:
    data_list = [ast.literal_eval(item) for item in content.split('\n') if item.strip()]
except ValueError:
    print("Error: File content is not in the expected format.")
    data_list = []



In [43]:
data_list

[('Raleigh School Bus', '27610'),
 ('Dub Farris Athletic Complex', '78249'),
 ('Legacy High School', '90280'),
 ('Montpelier School Bus', '05641'),
 ('North Florida Educational Institute', '32220'),
 ('Edmondson Heights Elementary School', '21207'),
 ('Fairview PreK-6 School', '45406'),
 ('Navarro High School', '78758'),
 ('Greenberg Elementary School', '93727'),
 ('Fred Moore High School', '76205'),
 ('Cleveland Central Catholic High School', '44105'),
 ('Bayless Elementary School', '63123'),
 ('Darlington-Lee Adult Education Campus', '29010'),
 ('Thibodaux High Stadium', '70301'),
 ('Savannah High School', '31404'),
 ('Wasilla School Bus', '99654'),
 ('Stanton Elementary School', '20020'),
 ('Gilmor Elementary School', '21213'),
 ('Mary Church Terrell School', '44111'),
 ('Westwood Elementary School', '37355'),
 ('Sweetwater High School', '37874'),
 ('Milwaukee Lutheran High School', '53222'),
 ('James McHenry Elementary School', '21223'),
 ('Booker T. Washington School', '61820'),
 

In [142]:
with open('cap_urls.txt', 'w') as f:
    for line in urls:
        f.write(f"{line}\n")

In [141]:
with open('failed_schools2.txt', 'w') as f:
    for line in failed_schools:
        f.write(f"{line}\n")

In [75]:
file_path = 'failed_schools2.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

    # Strip newline characters from each line and create a list
fs = [ast.literal_eval(line.strip()) for line in lines]

In [76]:
fs

[('Raleigh School Bus', '27610'),
 ('Dub Farris Athletic Complex', '78249'),
 ('Legacy High School', '90280'),
 ('Montpelier School Bus', '05641'),
 ('North Florida Educational Institute', '32220'),
 ('Edmondson Heights Elementary School', '21207'),
 ('Fairview PreK-6 School', '45406'),
 ('Navarro High School', '78758'),
 ('Greenberg Elementary School', '93727'),
 ('Fred Moore High School', '76205'),
 ('Cleveland Central Catholic High School', '44105'),
 ('Bayless Elementary School', '63123'),
 ('Darlington-Lee Adult Education Campus', '29010'),
 ('Thibodaux High Stadium', '70301'),
 ('Savannah High School', '31404'),
 ('Wasilla School Bus', '99654'),
 ('Stanton Elementary School', '20020'),
 ('Gilmor Elementary School', '21213'),
 ('Mary Church Terrell School', '44111'),
 ('Westwood Elementary School', '37355'),
 ('Milwaukee Lutheran High School', '53222'),
 ('James McHenry Elementary School', '21223'),
 ('Booker T. Washington School', '61820'),
 ('Lincoln Middle School', '20012'),
 (

In [61]:
doubles2 = ['https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Sweetwater+High+School&Zip=37874&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=470300001199',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Ruston+High+School&Zip=71270&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=220099000752',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Westinghouse+Academy&Zip=15208&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=421917007412',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Gila+Ridge+High+School&Zip=85365&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=040963002775',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Battle+Creek+High+School&Zip=49017&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=260000503830',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Bountiful+High+School&Zip=84010&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=490021000105',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Madison+Park+Academy&Zip=94621&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=062805004278',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Central+High+School&Zip=02908&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=440090000217',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Eagle+Academy&Zip=20024&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=110004900434',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Carroll+High+School&Zip=71203&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=220108000787',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Blytheville+High+School&Zip=72315&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=050332001578',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Oxford+High+School&Zip=48371&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=262724008184',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=West+Middle+School&Zip=61103&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=173451005448',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Union+High+School&Zip=74133&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=403060001704',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Ben+Davis+High+School&Zip=46214&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=181281002028',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Newton+Elementary+School&Zip=39345&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=280318001137',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Heritage+High+School&Zip=23605&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=510264001451',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Harrisburg+High+School&Zip=17104&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=421158006333',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=West+Middle+School&Zip=61103&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=173451005448',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=San+Diego+High+School&Zip=92101&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=063432011153',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Roosevelt+High+School&Zip=55406&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=272124001026',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Rochester+Prep+Charter+School&Zip=14604&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=360115306592',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Canyon+del+Oro+High+School&Zip=85737&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=040068000033',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Frost+Elementary+School&Zip=60056&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=171377001717',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Decatur+High+School&Zip=35601&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=010117002175',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Mountain+Brook+High+School&Zip=35223&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=010249001056',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Achievement+Academy&Zip=21214&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=240009000236',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Prescott++High+School&Zip=71857&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=051182000899',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Chamberlain+High+School&Zip=57325&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=461200000119',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Fowlerville+High+School&Zip=48836&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=261473005170',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Dalton+High+School&Zip=30720&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=130162000786',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Dearborn+High+School&Zip=48128&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=261160008095',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=NET+Charter+High+School&Zip=70113&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=220018802340',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=North+Little+Rock+High+School&Zip=72114&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=051068001566',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Spanish+Fork+High+School&Zip=84660&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=490063000399',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Bayless+High+School&Zip=63103&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=290450000715',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Independence+High+School&Zip=85303&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=040345000284',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Corona+del+Sol+High+School&Zip=85284&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=040834000790',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Normal+Community+High+School&Zip=61761&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=172862003002',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Gardena+High+School&Zip=90248&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=062271003034',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Weston+High+School&Zip=53924&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=551647002156',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Vicksburg+High+School&Zip=39180&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=280447000830',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Gardena+High+School&Zip=90248&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=062271003034',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Wahluke+High+School&Zip=99349&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=530936002070',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Largo+High+School&Zip=20774&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=240051001089',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=North+Miami+Senior+High+School&Zip=33161&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=120039000610',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Marshall+High+School&Zip=22043&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=510126000535',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Pearl+High+School&Zip=39208&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=280352000003',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Talladega+High+School&Zip=35160&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=010318001215',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Redford+High+School&Zip=48219&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=262946006525',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Jordan+High+School&Zip=90805&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=062250002728',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Hollywood+High+School&Zip=90028&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=062271003088',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Largo+High+School&Zip=20774&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=240051001089',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Lindhurst+High+School&Zip=95961&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=062409003630',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Roosevelt+High+School&Zip=90033&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=062271012791',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Central+High+School&Zip=02903&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=440090000217',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Dysart+High+School&Zip=85335&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=040269000198',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Redford+High+School&Zip=48219&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=262946006525',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Carver+High+School&Zip=30315&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=130012004223',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Southeast+Polk+High+School&Zip=50009&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=192682001572',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Compton+High+School&Zip=90220&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=060962013944',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=North+Miami+Senior+High+School&Zip=33161&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=120039000610',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Huntsville+High+School&Zip=35801&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=010180000631',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Lorain+High+School&Zip=44052&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=390442605326',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Barstow+High+School&Zip=92311&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=060402000355',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Largo+High+School&Zip=20774&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=240051001089',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Meridian+High+School&Zip=83642&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=160210000372',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Seminole+High+School&Zip=33776&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=120156001664',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Jordan+High+School&Zip=90002&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=062271003109']

In [62]:
doubles2 

['https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Sweetwater+High+School&Zip=37874&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=470300001199',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Ruston+High+School&Zip=71270&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=220099000752',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Westinghouse+Academy&Zip=15208&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=421917007412',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Gila+Ridge+High+School&Zip=85365&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=040963002775',
 'https://nces.ed.gov/ccd/schoolse

In [85]:
def remove_bus_from_tuple(tup):
    modified_first_element = tup[0].replace(' Bus', '').replace('School', '').replace('High', '')
        
    return (modified_first_element, tup[1])




failed_schools3 = [remove_bus_from_tuple(tup) for tup in fs]

        

In [86]:
failed_schools3

[('Raleigh ', '27610'),
 ('Dub Farris Athletic Complex', '78249'),
 ('Legacy  ', '90280'),
 ('Montpelier ', '05641'),
 ('North Florida Educational Institute', '32220'),
 ('Edmondson Heights Elementary ', '21207'),
 ('Fairview PreK-6 ', '45406'),
 ('Navarro  ', '78758'),
 ('Greenberg Elementary ', '93727'),
 ('Fred Moore  ', '76205'),
 ('Cleveland Central Catholic  ', '44105'),
 ('Bayless Elementary ', '63123'),
 ('Darlington-Lee Adult Education Campus', '29010'),
 ('Thibodaux  Stadium', '70301'),
 ('Savannah  ', '31404'),
 ('Wasilla ', '99654'),
 ('Stanton Elementary ', '20020'),
 ('Gilmor Elementary ', '21213'),
 ('Mary Church Terrell ', '44111'),
 ('Westwood Elementary ', '37355'),
 ('Milwaukee Lutheran  ', '53222'),
 ('James McHenry Elementary ', '21223'),
 ('Booker T. Washington ', '61820'),
 ('Lincoln Middle ', '20012'),
 ('Premier  ', '72114'),
 ('Mount Carmel  ', '60637'),
 ('Brookhaven ', '39601'),
 ('North Hill Elementary ', '98148'),
 ('O’Bannon  ', '38701'),
 ('Ranchos Eleme

In [113]:
len(urls)

415

In [155]:
urls_by_hand = pd.read_csv('/home/jklingl/Downloads/urls by hand - Sheet1.csv')

In [159]:
hand_urls = urls_by_hand.dropna()

In [163]:
for i in hand_urls['Urls']:
    print(i)

https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Legacy&Zip=90280&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=062271010863
https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=FairView+Elementary&Zip=45406&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=390438400789
https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Thibodaux&Miles=10&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=220090000725
https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=savannah+high+&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=130102004196
https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Booker+T.+Washington+&SchoolType=1&SchoolType=2&SchoolType

In [164]:
# most up to data script 
all_data2 = []
failed_urls = []

for i in hand_urls['Urls']:
    try:
        response = requests.get(i)
        soup = BeautifulSoup(response.content, 'html.parser')
        tables = soup.find_all('table')
        rows = tables[2].find_all('tr')
        data_dict = {}
        for row in rows:
            columns = row.find_all('td')
            if columns:
                x = columns[0].text.split(":")
                x = [item.strip() for item in x]
                if len(x)>=2:
                    key = x[0]
                    value = x[1]
                    data_dict[key] = value
                else:
                    pass
        # below gets the students, teachers, ratio
        rows2 = tables[6].find_all('tr')
        for row in rows2:
            col_name = row.find_all('td')[0].text.strip()
            value = row.find_all('td')[2].text.strip()
            data_dict[col_name] = value
        nces = []
        for x in tables[2].find('tr').find_all('td'):
            nces.append(x.text.strip())


        for item in nces:
            if 'NCES School ID' in item:
                nces_id = item.split(':')[-1].strip()
                nces_id = int(nces_id)
                data_dict['NCES School ID'] = nces_id
            elif 'State School ID' in item:
                state_id = item.split(':')[-1].strip()
                data_dict['State School ID'] = state_id

        # getting the grades and their enrollment numbers
        test = []
        for i in range(len(tables[7].find('tr').find_all('td'))):
            test.append(tables[7].find_all('tr')[0].find_all('td')[i].text)
        #pattern = re.compile('^\d+$')
        pattern = re.compile(r'\b(PK?|KG?|^\d+$)\b')
        # Extract numbers from each string in the list
        cleaned_numbers = [match.group() for item in test for match in re.finditer(pattern, item)]
        cleaned_numbers = fix_grades(cleaned_numbers)

        # splitting the list in half because first half is the grade and second half is the value
        midpoint = len(cleaned_numbers) //2

        grade = cleaned_numbers[:midpoint]
        enrollment = cleaned_numbers[midpoint:]
        for i, grade in enumerate(grade):
            data_dict[f'Grade {grade}'] = enrollment[i]

        # Getting races and their values
        race_num = []
        for i in range(len(tables[12].find_all('tr')[0].find_all('td'))):
            race_num.append((tables[12].find_all('tr')[1].find_all('td')[i].text))

        race = []
        for i in range(len(tables[12].find_all('tr')[0].find_all('td'))):
            race.append((tables[12].find_all('tr')[0].find_all('td')[i].text))

        clean_race = []
        for i in race_num:
            clean_race.append(i.strip())
        race_num = clean_race[1:]
        race = race[1:]
        #race_data = {race: int(count) for race, count in zip(race, race_num)}
        for i, race in enumerate(race):
            data_dict[race] = race_num[i]
        # Getting genders and     
        for i in range(len(tables[13].find_all('tr')[0].find_all('td'))):
            datastr = ((tables[13].find_all('tr')[1].find_all('td')[i].text))

        data_list = [item.strip() for item in datastr.split('\n') if item.strip()]

        genders = data_list[0:2]
        gender_counts = data_list[3:5]
        for i, gender in enumerate(genders):
            data_dict[gender] = gender_counts[i]

        all_data2.append(data_dict)
        sleep(2)
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch data from URL: {i}. Error: {e}")
        failed_urls.append(i)
        continue
    except IndexError:
        print(f"IndexError occurred while processing data from URL: {i}.")
        failed_urls.append(i)
        continue
    except Exception as e:
        print(f"An unexpected error occurred while processing data from URL: {i}. Error: {e}")
        failed_urls.append(i)
        continue 

In [105]:
doubles2

['https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Sweetwater+High+School&Zip=37874&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=470300001199',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Ruston+High+School&Zip=71270&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=220099000752',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Westinghouse+Academy&Zip=15208&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=421917007412',
 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=Gila+Ridge+High+School&Zip=85365&Miles=20&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=040963002775',
 'https://nces.ed.gov/ccd/schoolse

In [165]:
len(all_data2)

305

In [166]:
pd.DataFrame.from_dict(all_data2)

,School Name,District Name,Mailing Address,Type,Supervisory Union #,Website,Total Students:,Classroom Teachers (FTE):,Student/Teacher Ratio:,NCES School ID,...,Two or MoreRaces,Male,Female,Grade PK,Grade KG,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5
0,International Studies Lrng Center at Legacy Hi...,Los Angeles Unifieddistrict information,"5225 Tweedy Blvd.South Gate, CA 90280-5332",Regular school,N/A,http,928,41.15,22.55,62271010863,...,‡,‡,‡,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fairview Elementary School,Dayton Citydistrict information,"2314 Elsmere AveDayton, OH 45406-5729",Regular school,N/A,NaN,472,29.00,16.28,390438400789,...,18,278,194,19,72,62,66,55,59,59
2,Thibodaux High School,Lafourche Parishdistrict information,"1355 Tiger DriveThibodaux, LA 70301",Regular school,N/A,NaN,"1,454",63.32,22.96,220090000725,...,52,735,719,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Savannah Early College High School,Savannah-chatham Countydistrict information,"400 Pennsylvania AveSavannah, GA 31404",Regular school,N/A,NaN,151,10.10,14.95,130102004196,...,9,58,93,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Booker T. Washington High School,Atlanta Public Schoolsdistrict information,"45 Whitehouse Dr SWAtlanta, GA 30314",Regular school,N/A,NaN,831,60.10,13.83,130012004174,...,11,409,422,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,Tomah Middle,Tomah Area School Districtdistrict information,"612 Hollister AveTomah, WI 54660-1960",Regular school,N/A,http,676,47.22,14.32,551491001938,...,49,324,352,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301,Hanes Magnet School,Winston Salem / Forsyth County Schoolsdistrict...,"2355 Pleasant StWinston Salem, NC 27107",Regular school,N/A,http,906,52.27,17.33,370150000608,...,34,493,413,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302,Cardozo Education Campus,District Of Columbia Public Schoolsdistrict in...,"1200 CLIFTON ST. NWWashington, DC 20009",Regular school,N/A,http,639,66.50,9.61,110003000082,...,2,364,275,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,The Academy @ Shawnee,Jefferson Countydistrict information,"4001 Herman StreetLouisville, KY 40212",Regular school,N/A,NaN,532,40.00,13.30,210299000777,...,38,308,224,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [167]:
nces_pt5_hand = pd.DataFrame.from_dict(all_data2)

In [37]:
nces_NEW

,School Name,District Name,Mailing Address,Type,Supervisory Union #,Total Students:,Classroom Teachers (FTE):,Student/Teacher Ratio:,NCES School ID,State School ID,...,Grade PK,Grade KG,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8
0,KIPP Indy Legacy High,Kipp Indy Legacy Highdistrict information,"2255 N Ralston AveIndianapolis, IN 46218",Regular school,N/A,377,27.00,13.96,180020602669,IN-9135-9138,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Freedom High,Burke County Schoolsdistrict information,"511 Independence BoulevardMorganton, NC 28655",Regular school,N/A,"1,414",73.62,19.21,370048000176,NC-120-318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Washington Elementary School,Lorain Citydistrict information,"1025 W 23rd StLorain, OH 44052-4615",Regular school,N/A,259,17.00,15.24,390442601203,OH-044263-039347,...,32,34,39,37,29,41,47,NaN,NaN,NaN
3,Donaldsonville High School,Ascension Parishdistrict information,"100 Tiger DriveDonaldsonville, LA 70734",Regular school,N/A,435,39.86,10.91,220009000036,LA-003-003003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Carver Vocational-Technical High,Baltimore City Public Schoolsdistrict information,"2201 Presstman StreetBaltimore, MD 21216-4106",Vocational school,N/A,"1,020",55.50,18.38,240009000168,MD-30-300454,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417,Governor Wallace Rider Farrington High School,Hawaii Department Of Educationdistrict informa...,"1564 North King StHonolulu, HI 96817",Regular school,N/A,"2,238",140.00,15.99,150003000221,HI-001-1-106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1418,West Leyden High School,Leyden Chsd 212district information,"1000 N Wolf RdNorthlake, IL 60164-1438",Regular school,N/A,"1,803",117.71,15.32,172274002522,IL-06-016-2120-16-0601621200002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1419,GRAND RAPIDS SENIOR HIGH,Grand Rapids Public School Districtdistrict in...,"800 CONIFER DR NWGRAND RAPIDS, MN 55744-2458",Regular school,N/A,"1,129",51.62,21.87,271317000935,MN-010318-010318360,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1420,South Philadelphia HS,Philadelphia City Sddistrict information,"2101 S Broad StPhiladelphia, PA 19148-2541",Regular school,N/A,611,51.03,11.97,421899003859,PA-126515001-3859,...,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
nces_df = pd.DataFrame.from_dict(all_data2)

In [40]:
nces_df_pt2.to_csv('nces_data.csv', index=False)  

In [116]:
nces_pt2.to_csv('nces_pt2.csv', index=False)  

In [146]:
nces_pt3.to_csv('nces_pt3.csv', index=False)  

In [154]:
nces_pt4.to_csv('nces_pt4.csv', index=False)  

In [168]:
nces_pt5_hand.to_csv('nces_pt5_hand.csv', index=False)  

In [41]:
nces_df

,School Name,District Name,Mailing Address,Type,Supervisory Union #,Total Students:,Classroom Teachers (FTE):,Student/Teacher Ratio:,NCES School ID,State School ID,...,Grade PK,Grade KG,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8
0,KIPP Indy Legacy High,Kipp Indy Legacy Highdistrict information,"2255 N Ralston AveIndianapolis, IN 46218",Regular school,N/A,377,27.00,13.96,180020602669,IN-9135-9138,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Freedom High,Burke County Schoolsdistrict information,"511 Independence BoulevardMorganton, NC 28655",Regular school,N/A,"1,414",73.62,19.21,370048000176,NC-120-318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Washington Elementary School,Lorain Citydistrict information,"1025 W 23rd StLorain, OH 44052-4615",Regular school,N/A,259,17.00,15.24,390442601203,OH-044263-039347,...,32,34,39,37,29,41,47,NaN,NaN,NaN
3,Donaldsonville High School,Ascension Parishdistrict information,"100 Tiger DriveDonaldsonville, LA 70734",Regular school,N/A,435,39.86,10.91,220009000036,LA-003-003003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Carver Vocational-Technical High,Baltimore City Public Schoolsdistrict information,"2201 Presstman StreetBaltimore, MD 21216-4106",Vocational school,N/A,"1,020",55.50,18.38,240009000168,MD-30-300454,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417,Governor Wallace Rider Farrington High School,Hawaii Department Of Educationdistrict informa...,"1564 North King StHonolulu, HI 96817",Regular school,N/A,"2,238",140.00,15.99,150003000221,HI-001-1-106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1418,West Leyden High School,Leyden Chsd 212district information,"1000 N Wolf RdNorthlake, IL 60164-1438",Regular school,N/A,"1,803",117.71,15.32,172274002522,IL-06-016-2120-16-0601621200002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1419,GRAND RAPIDS SENIOR HIGH,Grand Rapids Public School Districtdistrict in...,"800 CONIFER DR NWGRAND RAPIDS, MN 55744-2458",Regular school,N/A,"1,129",51.62,21.87,271317000935,MN-010318-010318360,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1420,South Philadelphia HS,Philadelphia City Sddistrict information,"2101 S Broad StPhiladelphia, PA 19148-2541",Regular school,N/A,611,51.03,11.97,421899003859,PA-126515001-3859,...,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
import pandas as pd
df = pd.read_fwf('/home/jklingl/Downloads/school_safety_3 (1).txt')
df.to_csv('UPDATED_SCHOOL_SAFETY.csv')

In [56]:
df = pd.read_csv('/home/jklingl/capstone/school_safety_3 (1).csv')

/home/jklingl/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3398: DtypeWarning: Columns (40,90,92,94,96,98,100,102,103,162,164,211,212,213,214,215,216,217,218,219,220,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,566,567,568,569,570,571,572,573,574,575,576,577,750,751,752,753,754,755,756,757) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [46]:
txtfile = open('/home/jklingl/Downloads/school_safety_3 (1).txt')

In [52]:
df_empty = pd.DataFrame(list())
df_empty.to_csv('empty_csv.csv')

In [50]:
txtfile

<_io.TextIOWrapper name='/home/jklingl/Downloads/school_safety_3 (1).txt' mode='r' encoding='UTF-8'>

In [67]:
columns_to_drop = []
for i in df.columns:
    if i == 'School':
        pass
    elif i == 'Zip_Code':
        pass
    else:
        columns_to_drop.append(i)

In [68]:
columns_to_drop

['Unnamed: 0.2',
 'Unnamed: 0.1',
 'Unnamed: 0',
 'Incident_ID',
 'Month',
 'Day',
 'Year',
 'Date',
 'Victims_Killed',
 'Victims_Wounded',
 'Number_Victims',
 'Shooter_Killed',
 'Source',
 'Number_News',
 'Media_Attention',
 'Reliability',
 'Quarter',
 'City',
 'State',
 'School_Level',
 'Location',
 'Location_Type',
 'During_Classes',
 'Time_Period',
 'First_Shot',
 'Duration_min',
 'Summary',
 'Narrative',
 'Situation',
 'Targets',
 'Accomplice',
 'Accomplice_Narrative',
 'Hostages',
 'Barricade',
 'Officer_Involved',
 'Bullied',
 'Domestic_Violence',
 'Gang_Related',
 'Active_Shooter_FBI',
 'Shots_Fired',
 'LAT',
 'LNG',
 'Address',
 'Race_Asian',
 'Race_Black_African',
 'Race_Hawaiian_Pacific',
 'Race_Hispanic_Or_Latino',
 'Race_Native_Alaskan',
 'Race_White_Caucasian',
 'Race_Two_Or_More_Races',
 'Race_Some_Other_Race',
 'Sex_And_Age_Male_Under_5_Years',
 'Sex_And_Age_Female_Under_5_Years',
 'Sex_And_Age_Male_5_To_9_Years',
 'Sex_And_Age_Female_5_To_9_Years',
 'Sex_And_Age_Male_1

In [74]:
df.drop(columns = columns_to_drop, inplace = True)

KeyError: "['Unnamed: 0.2' 'Unnamed: 0.1' 'Unnamed: 0' ... 'Ancestry_Yakama'\n 'Ancestry_Soviet_Union' 'Immigration_Cabo_Verde'] not found in axis"

In [77]:
df.to_csv('zipcodes.csv', index=False)  

In [156]:
nces_df.to_csv('nces_NEW.csv', index=False)

Merging dfs

In [170]:
NCES_FINAL = pd.concat( 
    map(pd.read_csv, ['/home/jklingl/capstone/nces_data/nces_data.csv', 
                      '/home/jklingl/capstone/nces_data/nces_pt2.csv',
                      '/home/jklingl/capstone/nces_data/nces_pt3.csv',
                      '/home/jklingl/capstone/nces_data/nces_pt4.csv',
                      '/home/jklingl/capstone/nces_data/nces_pt5_hand.csv']), ignore_index=True) 


In [171]:
NCES_FINAL

,School Name,District Name,Mailing Address,Type,Supervisory Union #,Total Students:,Classroom Teachers (FTE):,Student/Teacher Ratio:,NCES School ID,State School ID,...,Grade PK,Grade KG,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8
0,KIPP Indy Legacy High,Kipp Indy Legacy Highdistrict information,"2255 N Ralston AveIndianapolis, IN 46218",Regular school,NaN,377,27.00,13.96,180020602669,IN-9135-9138,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Freedom High,Burke County Schoolsdistrict information,"511 Independence BoulevardMorganton, NC 28655",Regular school,NaN,"1,414",73.62,19.21,370048000176,NC-120-318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Washington Elementary School,Lorain Citydistrict information,"1025 W 23rd StLorain, OH 44052-4615",Regular school,NaN,259,17.00,15.24,390442601203,OH-044263-039347,...,32.0,34.0,39.0,37.0,29.0,41.0,47.0,NaN,NaN,NaN
3,Donaldsonville High School,Ascension Parishdistrict information,"100 Tiger DriveDonaldsonville, LA 70734",Regular school,NaN,435,39.86,10.91,220009000036,LA-003-003003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Carver Vocational-Technical High,Baltimore City Public Schoolsdistrict information,"2201 Presstman StreetBaltimore, MD 21216-4106",Vocational school,NaN,"1,020",55.50,18.38,240009000168,MD-30-300454,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263,Tomah Middle,Tomah Area School Districtdistrict information,"612 Hollister AveTomah, WI 54660-1960",Regular school,NaN,676,47.22,14.32,551491001938,WI-5747-0160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202.0,226.0,248.0
2264,Hanes Magnet School,Winston Salem / Forsyth County Schoolsdistrict...,"2355 Pleasant StWinston Salem, NC 27107",Regular school,NaN,906,52.27,17.33,370150000608,NC-340-392,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,320.0,292.0,294.0
2265,Cardozo Education Campus,District Of Columbia Public Schoolsdistrict in...,"1200 CLIFTON ST. NWWashington, DC 20009",Regular school,NaN,639,66.50,9.61,110003000082,DC-001-454,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,49.0,59.0
2266,The Academy @ Shawnee,Jefferson Countydistrict information,"4001 Herman StreetLouisville, KY 40212",Regular school,NaN,532,40.00,13.30,210299000777,KY-056275000-056275590,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,38.0,33.0


In [172]:
NCES_FINAL.to_csv('NCES_FINAL.csv', index=False)  

In [180]:
count = 0
for x in NCES_FINAL.duplicated():
    if x == True:
        count+=1
    else:
        pass
    
    

In [182]:
nces_final_clean = NCES_FINAL.drop_duplicates()

In [188]:
nces_final_clean.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2261    False
2263    False
2264    False
2266    False
2267    False
Length: 1959, dtype: bool

In [189]:
count = 0
for x in nces_final_clean.duplicated():
    if x == True:
        count+=1
    else:
        pass

In [184]:
nces_final_clean['State'] = nces_final_clean['State School ID'].str[:2]

/tmp/ipykernel_11585/3030438046.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nces_final_clean['State'] = nces_final_clean['State School ID'].str[:2]


In [186]:
nces_final_clean.to_csv('nces_final_clean.csv', index=False)  